In [2]:
import pandas as pd

In [3]:
from pathlib import Path

In [4]:
from concurrent.futures import ProcessPoolExecutor

In [5]:
project_dir = Path('.').resolve().parents[0]

In [6]:
data_dir = project_dir/'data'/'raw'

In [26]:
def read_csv(path):
    try:
        df = pd.read_csv(path, sep = '\t')
        df['parse_date'] = path.name.rstrip('.tsv')
        return df
    except:
        return None

In [27]:
with ProcessPoolExecutor() as pool:
    dfs = pool.map(read_csv, data_dir.iterdir())

In [28]:
df = pd.concat(dfs)

In [29]:
df.head(1)

,provider,provider_id,name,price,original_price,hidden_price,URL,parse_date
0,bol,9200000093063167,Philips Sonicare ProtectiveClean 4500 HX6830/4...,79.99,NaN,79.99,https://bol.com/nl/nl/p/philips-sonicare-prote...,2021-10-06 09:31AM


In [30]:
df['parse_date'] = pd.to_datetime(df['parse_date'])

In [31]:
df.set_index('parse_date', inplace = True)

In [32]:
df.sort_index(inplace = True)

In [33]:
def clean_numeric(s, to_type = float):
    if isinstance(s, str):
        return to_type(s.strip().strip('-'))
    else:
        return s

In [34]:
df['price'] = df['price'].apply(clean_numeric)

In [35]:
df.to_csv('combined_data.tsv', sep = '\t')

In [38]:
df[df.provider == 'mediamarkt']

,provider,provider_id,name,price,original_price,hidden_price,URL
parse_date,,,,,,,
2021-10-09 14:17:00,mediamarkt,MMNL1700318,LENOVO IdeaCentre 5 - i5-10400 8GB 512GB SSD,549.00,549.,NaN,https://mediamarkt.nl/nl/product/_lenovo-ideac...
2021-10-09 14:17:00,mediamarkt,MMNL1698756,MSI MAG META 5 3SI-441MYS,1139.00,1139.,NaN,https://mediamarkt.nl/nl/product/_msi-mag-meta...
2021-10-09 14:17:00,mediamarkt,MMNL1698956,HP OMEN 25L GT12-1410nd,1389.00,1389.,NaN,https://mediamarkt.nl/nl/product/_hp-omen-25l-...
2021-10-09 14:17:00,mediamarkt,MMNL1696650,HP OMEN 25L GT12-1420nd,1499.00,1499.,NaN,https://mediamarkt.nl/nl/product/_hp-omen-25l-...
2021-10-09 14:17:00,mediamarkt,MMNL1670320,"APPLE iMac 21.5"" - i5/8GB/1TB/FHD",999.00,1249.,NaN,https://mediamarkt.nl/nl/product/_apple-imac-2...
...,...,...,...,...,...,...,...
2021-10-09 17:22:00,mediamarkt,MMNL1698952,HP Pavilion TP01-2050nd,649.99,649.99,NaN,https://mediamarkt.nl/nl/product/_hp-pavilion-...
2021-10-09 17:22:00,mediamarkt,MMNL1698756,MSI MAG META 5 3SI-441MYS,1139.00,1139.,NaN,https://mediamarkt.nl/nl/product/_msi-mag-meta...
2021-10-09 17:22:00,mediamarkt,MMNL1701541,SCREENON GamePC V621817 - Ryzen 3 480GB,559.99,559.99,NaN,https://mediamarkt.nl/nl/product/_screenon-gam...


In [39]:
df[df.provider_id.isnull()]

,provider,provider_id,name,price,original_price,hidden_price,URL
parse_date,,,,,,,
